In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import requests
from tqdm import tqdm

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
line_df = pd.read_csv("/content/drive/MyDrive/ekidata/line20230105free.csv")
station_df = pd.read_csv("/content/drive/MyDrive/ekidata/station20230105free.csv")
join_df = pd.read_csv("/content/drive/MyDrive/ekidata/join20220921.csv")
pref_df = pd.read_csv("/content/drive/MyDrive/ekidata/pref_map.csv")

In [ ]:
line_df.head()

,line_cd,company_cd,line_name,line_name_k,line_name_h,line_color_c,line_color_t,line_type,lon,lat,zoom,e_status,e_sort
0,1001,3,中央新幹線,チュウオウシンカンセン,中央新幹線,NaN,NaN,NaN,137.493896,35.411438,8,1,1001
1,1002,3,東海道新幹線,トウカイドウシンカンセン,東海道新幹線,NaN,NaN,NaN,137.721489,35.144122,7,0,1002
2,1003,4,山陽新幹線,サンヨウシンカンセン,山陽新幹線,NaN,NaN,NaN,133.147896,34.419338,7,0,1003
3,1004,2,東北新幹線,トウホクシンカンセン,東北新幹線,NaN,NaN,NaN,140.763192,38.274267,7,0,1004
4,1005,2,上越新幹線,ジョウエツシンカンセン,上越新幹線,NaN,NaN,NaN,139.121488,36.798565,8,0,1005


In [ ]:
pref_df.head()

,pref_cd,pref_name
0,1,北海道
1,2,青森県
2,3,岩手県
3,4,宮城県
4,5,秋田県


In [ ]:
station_df.head()

,station_cd,station_g_cd,station_name,station_name_k,station_name_r,line_cd,pref_cd,post,address,lon,lat,open_ymd,close_ymd,e_status,e_sort
0,1110101,1110101,函館,NaN,NaN,11101,1,040-0063,北海道函館市若松町１２-１３,140.726413,41.773709,1902-12-10,0000-00-00,0,1110101
1,1110102,1110102,五稜郭,NaN,NaN,11101,1,041-0813,函館市亀田本町,140.733539,41.803557,0000-00-00,0000-00-00,0,1110102
2,1110103,1110103,桔梗,NaN,NaN,11101,1,041-0801,北海道函館市桔梗３丁目４１-３６,140.722952,41.846457,1902-12-10,0000-00-00,0,1110103
3,1110104,1110104,大中山,NaN,NaN,11101,1,041-1121,亀田郡七飯町大字大中山,140.713580,41.864641,0000-00-00,0000-00-00,0,1110104
4,1110105,1110105,七飯,NaN,NaN,11101,1,041-1111,亀田郡七飯町字本町,140.688556,41.886971,0000-00-00,0000-00-00,0,1110105


There are eki with the same name, like 東山

In [ ]:
station_df[station_df["station_name"] == "東山"]

,station_cd,station_g_cd,station_name,station_name_k,station_name_r,line_cd,pref_cd,post,address,lon,lat,open_ymd,close_ymd,e_status,e_sort
11,1110112,1110112,東山,NaN,NaN,11101,1,049-2141,茅部郡森町字駒ケ岳,140.605222,42.061720,1943-02-26,2017-03-04,2,1110112
6584,3101606,3101606,東山,NaN,NaN,31016,29,630-0225,生駒市東山町１１４４-１２,135.708500,34.648633,0000-00-00,0000-00-00,0,3101606
9343,9961110,9961110,東山,NaN,NaN,99611,26,605-0022,京都市東山区大井手町,135.780448,35.009425,0000-00-00,0000-00-00,0,9961110
9892,9970610,9970610,東山,NaN,NaN,99706,33,703-8291,岡山市徳吉町２-１０,133.942759,34.656245,0000-00-00,0000-00-00,0,9970610


In [ ]:
np.setdiff1d(np.unique(station_df["station_g_cd"]), np.unique(station_df["station_cd"]))

array([ 100915,  100917,  101009,  101010, 1140501, 1180430, 2400102,
       2400103])

In [ ]:
np.setdiff1d(np.unique(station_df["station_cd"]), np.unique(station_df["station_g_cd"]))

array([1110201, 1110301, 1110401, ..., 9992618, 9992620, 9992801])

there are station cd ids that are station_cd only or station_g_cd only.So I need both IDs in the analysis. 

In [ ]:
station_w_diff_cd = station_df[station_df["station_g_cd"] != station_df["station_cd"]]

gcd_cd = {}

for i, row in station_w_diff_cd.iterrows():
    gcd_cd[row["station_g_cd"]] = row["station_cd"]

In [ ]:
join_df.head()

,line_cd,station_cd1,station_cd2
0,1002,100201,100202
1,1002,100202,100203
2,1002,100203,100204
3,1002,100204,100205
4,1002,100205,100206


replace the unique station_g_cd with station_cd

In [ ]:
100917 in join_df["station_cd1"].unique()

True

In [ ]:
join_df.station_cd1.replace(gcd_cd, inplace=True)

join_df.station_cd2.replace(gcd_cd, inplace=True)

In [ ]:
100917 in join_df["station_cd1"].unique()

False

In [ ]:
join_eki = np.unique(np.concatenate(([join_df["station_cd1"], join_df["station_cd2"]])))

station_df[~station_df["station_cd"].isin(join_eki)]

,station_cd,station_g_cd,station_name,station_name_k,station_name_r,line_cd,pref_cd,post,address,lon,lat,open_ymd,close_ymd,e_status,e_sort
0,1110101,1110101,函館,NaN,NaN,11101,1,040-0063,北海道函館市若松町１２-１３,140.726413,41.773709,1902-12-10,0000-00-00,0,1110101
1,1110102,1110102,五稜郭,NaN,NaN,11101,1,041-0813,函館市亀田本町,140.733539,41.803557,0000-00-00,0000-00-00,0,1110102
11,1110112,1110112,東山,NaN,NaN,11101,1,049-2141,茅部郡森町字駒ケ岳,140.605222,42.061720,1943-02-26,2017-03-04,2,1110112
12,1110113,1110113,姫川,NaN,NaN,11101,1,049-2306,茅部郡森町字姫川,140.591632,42.081312,1951-05-19,2017-03-04,2,1110113
13,1110114,1110114,池田園,NaN,NaN,11101,1,041-1352,亀田郡七飯町字軍川,140.700333,41.990692,1945-06-01,2022-03-12,2,1110114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10823,9992505,9992505,朝日通,NaN,NaN,99925,46,892-0821,鹿児島市名山町,130.557286,31.594374,0000-00-00,0000-00-00,0,9992505
10824,9992506,9992506,いづろ通,NaN,NaN,99925,46,892-0828,鹿児島市金生町,130.557672,31.591824,0000-00-00,0000-00-00,0,9992506
10825,9992507,9992507,天文館通,NaN,NaN,99925,46,892-0843,鹿児島市千日町,130.555075,31.590691,0000-00-00,0000-00-00,0,9992507
10826,9992508,9992508,高見馬場,NaN,NaN,99925,46,892-0844,鹿児島市山之口町,130.550977,31.588726,0000-00-00,0000-00-00,0,9992508


In [ ]:
station_df[~station_df["station_g_cd"].isin(join_eki)]

,station_cd,station_g_cd,station_name,station_name_k,station_name_r,line_cd,pref_cd,post,address,lon,lat,open_ymd,close_ymd,e_status,e_sort
0,1110101,1110101,函館,NaN,NaN,11101,1,040-0063,北海道函館市若松町１２-１３,140.726413,41.773709,1902-12-10,0000-00-00,0,1110101
1,1110102,1110102,五稜郭,NaN,NaN,11101,1,041-0813,函館市亀田本町,140.733539,41.803557,0000-00-00,0000-00-00,0,1110102
11,1110112,1110112,東山,NaN,NaN,11101,1,049-2141,茅部郡森町字駒ケ岳,140.605222,42.061720,1943-02-26,2017-03-04,2,1110112
12,1110113,1110113,姫川,NaN,NaN,11101,1,049-2306,茅部郡森町字姫川,140.591632,42.081312,1951-05-19,2017-03-04,2,1110113
13,1110114,1110114,池田園,NaN,NaN,11101,1,041-1352,亀田郡七飯町字軍川,140.700333,41.990692,1945-06-01,2022-03-12,2,1110114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10851,9992608,9992508,高見馬場,NaN,NaN,99926,46,892-0844,鹿児島市山之口町,130.550977,31.588726,0000-00-00,0000-00-00,0,9992608
10854,9992611,1190411,鹿児島中央駅前,NaN,NaN,99926,46,890-0053,鹿児島市中央町３７-１,130.543703,31.584120,0000-00-00,0000-00-00,0,9992611
10861,9992618,1193002,純心学園前,NaN,NaN,99926,46,890-0065,鹿児島市郡元１,130.542694,31.566872,0000-00-00,0000-00-00,0,9992618
10863,9992620,9992516,郡元,NaN,NaN,99926,46,890-0063,鹿児島県鹿児島市鴨池一丁目,130.549024,31.562466,0000-00-00,0000-00-00,0,9992620


In [ ]:
pref_id_name = {}
for i, row in pref_df.iterrows():
    pref_id_name[row["pref_cd"]] = row["pref_name"]

In [ ]:
station_id_detail = {}

target_pref = ["北海道"]

###Use your Google Earth Engine ngrok url here
url = "[Your GEE ngrok url]"

for i, row in tqdm(station_df.iterrows()):
    station_cd = row["station_cd"]
    #station_g_cd = row["station_g_cd"]
    lat = row["lat"]
    lng = row["lon"]
    name = row["station_name"]
    #close_ymd = row["close_ymd"]
    pref_cd = row["pref_cd"]

    pref_name = pref_id_name[pref_cd]

    if pref_name in target_pref:

        api_url = f"{url}/worldpop?lat={lat}&lon={lng}&start_date=2020-01-01&end_date=2021-01-01"
        result = requests.get(api_url).json()["result"]
        
        pop = None
        if result and "population_mean" in result:
            pop = result["population_mean"]

        elevation_url = f"{url}/elevation?lat={lat}&lon={lng}"

        elevation = None

        result = requests.get(elevation_url).json()["result"]

        if result and "elevation" in result:
            elevation = result["elevation"]

        #print (pop, elevation)
        station_id_detail[station_cd] = {"name": name, "lng": lng, "lat": lat, "pref": pref_name, "pop": pop, "elevation": elevation}
        
        #station_id_detail[station_g_cd] = {"name": name, "lng": lng, "lat": lat, "close_ymd": close_ymd, "pref": pref_id_name[pref_cd]}

10887it [13:27, 13.48it/s] 


In [ ]:
len(station_id_detail)

651

In [ ]:
line_id_detail = {}
for i, row in line_df.iterrows():
    id = row["line_cd"]
    name = row["line_name"]

    line_id_detail[id] = name

In [ ]:
station_connected = set()

connections = "START_ID,END_ID,line,name\n"

for i, row in join_df.iterrows():
    line_id = row["line_cd"]

    station_1 = row["station_cd1"]
    station_2 = row["station_cd2"]

    if station_1 in station_id_detail and station_2 in station_id_detail:

        #station_1_name = station_id_detail[station_1]["name"]
        #station_2_name = station_id_detail[station_2]["name"]

        line_name = line_id_detail[line_id]


        station_connected.add(station_1)
        station_connected.add(station_2)

        connections += f"{station_1},{station_2},{line_id},{line_name}\n" 

In [ ]:
len(station_connected)

456

In [ ]:
len(connections.split("\n"))

495

In [ ]:
with open("connections.csv", "w") as output:
    output.write(connections)

In [ ]:
station_temp = "id,lat,lng,name,pref,population,elevation\n"
#station_temp = "id:ID(STATION),lat:double,lng:double,name:string,pref:string,close_ymd:date\n"

for station_cd in station_id_detail:
    if station_cd in station_connected:
        station_temp += f"{station_cd},{station_id_detail[station_cd]['lat']},{station_id_detail[station_cd]['lng']},{station_id_detail[station_cd]['name']},{station_id_detail[station_cd]['pref']},{round(station_id_detail[station_cd]['pop'],2)},{station_id_detail[station_cd]['elevation']}\n"


with open("stations.csv", "w") as output:
    output.write(station_temp)

In [ ]:
!pip install gqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
station_df_csv = pd.read_csv("stations.csv")
connections_df = pd.read_csv("connections.csv")

In [ ]:
station_df_csv.head()

,id,lat,lng,name,pref,population,elevation
0,1110103,41.846457,140.722952,桔梗,北海道,20.43,37
1,1110104,41.864641,140.713580,大中山,北海道,8.46,35
2,1110105,41.886971,140.688556,七飯,北海道,9.99,33
3,1110106,41.905400,140.646525,新函館北斗,北海道,6.73,30
4,1110107,41.930011,140.635183,仁山,北海道,0.86,91


In [ ]:
connections_df.head()

,START_ID,END_ID,line,name
0,9910617,1111921,11101,JR函館本線(函館～長万部)
1,1111921,1110103,11101,JR函館本線(函館～長万部)
2,1110103,1110104,11101,JR函館本線(函館～長万部)
3,1110104,1110105,11101,JR函館本線(函館～長万部)
4,1110105,1110106,11101,JR函館本線(函館～長万部)


In [ ]:
join_eki = np.unique(np.concatenate(([connections_df["START_ID"], connections_df["END_ID"]])))

In [ ]:
len(join_eki)

456

In [ ]:
len(connections_df)

493

In [ ]:
temp = connections_df.groupby(['START_ID','END_ID']).size().reset_index().rename(columns={0:'count'})

In [ ]:
temp[temp["count"] > 1]

,START_ID,END_ID,count
291,1111602,1111603,2
348,1111910,1111911,2
349,1111911,1111912,2
350,1111912,1111913,2
351,1111913,1111914,2
352,1111914,1111915,2
353,1111915,1111916,2
354,1111916,1111917,2
355,1111917,1111918,2
356,1111918,1111919,2


CREATE CONSTRAINT ON (s:Station) ASSERT s.name IS UNIQUE;

In [ ]:
from gqlalchemy import Memgraph

###You can get this information from Memgraph Cloud's page
MEMGRAPH_HOST = '[YOUR MEMGRAPH CLOUD HOST]'
MEMGRAPH_PORT = 7687
MEMGRAPH_USERNAME = '[YOUR MEMGRAPH CLOUD USERNAME]'
# Place your Memgraph password that was created during Project creation
MEMGRAPH_PASSWORD = '[YOUR MEMGRAPH CLOUD PASSWORD]'


In [ ]:


connection = Memgraph(MEMGRAPH_HOST, MEMGRAPH_PORT, MEMGRAPH_USERNAME, MEMGRAPH_PASSWORD, encrypted=True)

for i, row in station_df_csv.iterrows():
    results = connection.execute(
        f"MERGE (:Station {{id: {row['id']}, name: '{row['name']}', lat: {row['lat']}, lng: {row['lng']}, pref: '{row['pref']}', population: {row['population']}, elevation: {row['elevation']} }});\n"
        #'CREATE (n:FirstNode { message: "Hello Memgraph from Python!" }) RETURN n.message AS message'
    )
    #print("Created node with message:", next(results)["message"])

In [ ]:


connection = Memgraph(MEMGRAPH_HOST, MEMGRAPH_PORT, MEMGRAPH_USERNAME, MEMGRAPH_PASSWORD, encrypted=True)

for i, row in connections_df.iterrows():
    results = connection.execute(
        f"MATCH (u:Station), (v:Station) WHERE u.id = {row['START_ID']} AND v.id = {row['END_ID']} CREATE (u)-[:CONNECTS {{line: '{row['line']}', name: '{row['name']}'}}]->(v);"
        #f"MATCH (a:Station {{id: '{row['START_ID']}'}}) MATCH (b:Station {{id: '{row['END_ID']}'}}) MERGE (a)-[:CONNECTS {{line: '{row['line']}', name: '{row['name']}'}}]->(b);\n"
        #'CREATE (n:FirstNode { message: "Hello Memgraph from Python!" }) RETURN n.message AS message'
    )
    #print("Created node with message:", next(results)["message"])